In [309]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

## memory 공유 확인 위한 dummy chain 생성

In [216]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

dummy_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
dummy_prompt.pretty_print()

================================ System Message ================================


            You are a robot that classifies customer input messages into specific types. If order_id is provided, you need to output '조회'.
            If order_id is not provided, you need to classify the customer input message into one of the following two types:
            - 상품 문의, 주문 내역 조회, 주문 변경 내역 조회, 주문 취소 내역 조회: '문의'
            - 주문 요청, 주문 변경 요청, 주문 취소 요청: '요청'
            

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

input:{input}
order_id:{order_id}


In [117]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI()

In [118]:
dummy_chain = dummy_prompt | model 

In [119]:
from langchain_core.runnables.history import RunnableWithMessageHistory

dumy_chain_with_memory = RunnableWithMessageHistory(
    dummy_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [310]:
response = dumy_chain_with_memory.invoke(
    {"input": "주문 변경하고 싶어", 
    "order_id": None},
    config={"configurable": {"session_id": "test_240517-2"}}
    )
response

Parent run a727ab67-dfc3-4e5b-bd65-802e9f19b9b0 not found for run cec82afd-a02a-4857-8e4f-5c176776f5cd. Treating as a root run.


AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5af9ca04-d147-47a9-9c23-5f3f258128cb-0')

In [113]:
store["test_240517-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-da099424-affc-4335-b10e-8c6bb4093e93-0'),
 HumanMessage(content='없음'),
 AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 89, 'total_tokens': 94}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d3e2950d-3111-40bc-8ff3-7a1ac5996cf3-0'),
 HumanMessage(content='없음'),
 AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 292, 'total_tokens': 293}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0aeac858-9413-4f2a-821e-f779e312e018-0')]

In [311]:
store["test_240517-2"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5af9ca04-d147-47a9-9c23-5f3f258128cb-0')]

## 주문 변경/취소 분류 체인

이젠 필요 없음

In [ ]:
chat_history = store["test_240516-1"].messages
chat_history 

In [183]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            you need to review the chat_history from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
order_change_cancel_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the chat_history from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])

프롬프트 수정
* You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.

In [220]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
order_change_cancel_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])

프롬프트 수정2

In [312]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.

            Classify as '주문 변경' if there are any indications or keywords related to modifying an order, such as:
            - 변경
            - 수정
            - 바꾸다
            - 변경하고 싶어

            Classify as '주문 취소' if there are any indications or keywords related to canceling an order, such as:
            - 취소
            - 취소하고 싶어
            - 취소 요청
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
order_change_cancel_prompt


ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n\n            Classify as '주문 변경' if there are any indications or keywords related to modifying an order, such as:\n            - 변경\n            - 수정\n            - 바꾸다\n            - 변경하고 싶어\n\n            Classify as '주문 취소' if there are any indications or keywords related to can

In [313]:
classify_change_or_cancel_chain = order_change_cancel_prompt | model
classify_change_or_cancel_chain

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n\n            Classify as '주문 변경' if there are any indications or keywords related to modifying an order, such as:\n            - 변경\n            - 수정\n            - 바꾸다\n            - 변경하고 싶어\n\n            Classify as '주문 취소' if there are any indications or keywords related to can

오늘 추가 부분

In [314]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_change_or_cancel_chain_with_memory = RunnableWithMessageHistory(
    classify_change_or_cancel_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

최초 지정 session_id

In [262]:
from langchain_core.runnables import RunnableLambda
classify_change_or_cancel_chain_with_memory_lambda = RunnableLambda(
    lambda x: classify_change_or_cancel_chain_with_memory.invoke(x,
                                                                 config={"configurable": {"session_id": "test_240517-1"}})
)
classify_change_or_cancel_chain_with_memory_lambda

RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-1'}}))

session_id 변경

In [315]:
from langchain_core.runnables import RunnableLambda
classify_change_or_cancel_chain_with_memory_lambda = RunnableLambda(
    lambda x: classify_change_or_cancel_chain_with_memory.invoke(x,
                                                                 config={"configurable": {"session_id": "test_240517-2"}})
)
classify_change_or_cancel_chain_with_memory_lambda

RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-2'}}))

## 승인 메시지 식별 체인

In [264]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

classify_confirmation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.
            
            To make this determination, the following conditions must be met:
            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.
            2. The user must have explicitly expressed consent in response to the AIMessage asking for approval of the action specified in action_type.
            
            Your response must be either 'yes' or 'no'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}\naction_type: {action_type}"),
    ]
)
classify_confirmation_prompt

ChatPromptTemplate(input_variables=['action_type', 'chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.\n            2. The user m

In [265]:
classify_confirmation_chain = classify_confirmation_prompt | model 
classify_confirmation_chain

ChatPromptTemplate(input_variables=['action_type', 'chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to determine whether the user has responded to the AI's request for approval regarding the action specified in action_type.\n            \n            To make this determination, the following conditions must be met:\n            1. There must be an AIMessage that asks for approval for the action specified in action_type, based on the specified order details.\n            2. The user m

In [266]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_confirmation_chain_with_memory = RunnableWithMessageHistory(
    classify_confirmation_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

최초 지정 session_id

In [267]:
from langchain_core.runnables import RunnableLambda
classify_confirmation_chain_with_memory_lambda = RunnableLambda(
    lambda x: classify_confirmation_chain_with_memory.invoke(x,
                                                             config={"configurable": {"session_id": "test_240517-1"}})
)
classify_confirmation_chain_with_memory_lambda

RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-1'}}))

session_id 변경

In [268]:
from langchain_core.runnables import RunnableLambda
classify_confirmation_chain_with_memory_lambda = RunnableLambda(
    lambda x: classify_confirmation_chain_with_memory.invoke(x,
                                                             config={"configurable": {"session_id": "test_240517-2"}})
)
classify_confirmation_chain_with_memory_lambda

RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-2'}}))

## action_type 추가 위한 보조 함수

In [269]:
def helper(dict):
    inputs = dict["inputs"]
    action_type =  dict["action_type"]
    # action_type = action_type.content
    inputs["action_type"] = action_type
    return inputs

## 종합-입력에 승인 메시지 포함 여부까지 추가

구 버전

In [270]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

confirmation_chain = (
    {
        "inputs": RunnablePassthrough(),
        "action_type": classify_change_or_cancel_chain
    } | RunnableLambda(helper) | RunnablePassthrough.assign(execution_confirmation=classify_confirmation_chain)
    )
confirmation_chain

{
  inputs: RunnablePassthrough(),
  action_type: ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the chat_history from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n\n            Classify as '주문 변경' if there are any indications or keywords related to modifying an order, such as:\n            - 변경\n            - 수정\n            - 바꾸다\n            - 변경하고 싶어\n\n            Classify as '주문 취소' if there are any indications 

수정 버전

In [316]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

confirmation_chain = (
    {
        "inputs": RunnablePassthrough(),
        "action_type": classify_change_or_cancel_chain_with_memory_lambda
    } 
    | RunnableLambda(helper) 
    | RunnablePassthrough.assign(execution_confirmation=classify_confirmation_chain_with_memory_lambda)
    )
confirmation_chain


{
  inputs: RunnablePassthrough(),
  action_type: RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-2'}}))
}
| RunnableLambda(helper)
| RunnableAssign(mapper={
    execution_confirmation: RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-2'}}))
  })

구 버전

In [272]:
response = confirmation_chain.invoke(
    {"chat_history": chat_history,
     "input": None,
     "order_id": 7}
)
response

NameError: name 'chat_history' is not defined

수정 버전

In [318]:
store["test_240517-2"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5af9ca04-d147-47a9-9c23-5f3f258128cb-0')]

In [317]:
response = confirmation_chain.invoke(
    {"input": None,
     "order_id": 7}
)
response

Parent run ff430757-7942-4fd6-b5a6-0fafe419fb19 not found for run 28fbfe1e-c329-49eb-b134-956b1778645b. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got None.')
Parent run 1764daaf-56eb-40f5-aa4f-49582b93c49e not found for run 1a2a1d1c-b058-42d3-8d24-e828bc4b868c. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got None.')


{'input': None,
 'order_id': 7,
 'action_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 193, 'total_tokens': 196}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ceea037a-8846-4d02-9c22-f0c41c4f6fdb-0'),
 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 269, 'total_tokens': 270}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-36b741ed-50b6-4c75-a9c5-d797c8e39fae-0')}

In [319]:
store["test_240517-2"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5af9ca04-d147-47a9-9c23-5f3f258128cb-0')]

## 승인 메시지 작성 체인

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

generate_confirm_message_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that generates a confirmation request message based on the action_type and queried_result.
            create a message to show the queried_result and ask for final confirmation considering action_type.
            The response should be generated in Korean.
            """
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "action_type:{action_type}\nqueried_result:{queried_result}")
    ]
)
generate_confirm_message_prompt

ChatPromptTemplate(input_variables=['action_type', 'queried_result'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            You are a robot that generates a confirmation request message based on the action_type and queried_result.\n            create a message to show the queried_result and ask for final confirmation considering action_type.\n            The response should be generated in Korean.\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['action_type', 'queried_result'], template='action_type:{action_type}\nqueried_result:{queried_result}'))])

In [ ]:
generate_confirm_message_chain = generate_confirm_message_prompt | model
generate_confirm_message_chain

ChatPromptTemplate(input_variables=['action_type', 'queried_result'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='\n            You are a robot that generates a confirmation request message based on the action_type and queried_result.\n            create a message to show the queried_result and ask for final confirmation considering action_type.\n            The response should be generated in Korean.\n            ')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['action_type', 'queried_result'], template='action_type:{action_type}\nqueried_result:{queried_result}'))])
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001C1117B1B40>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001C1100FF400>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

generate_confirm_message_chain_with_memory = RunnableWithMessageHistory(
    generate_confirm_message_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
from langchain_core.runnables import RunnableLambda
generate_confirm_message_chain_with_memory_lambda = RunnableLambda(
    lambda x: generate_confirm_message_chain_with_memory.invoke(x,
                                                             config={"configurable": {"session_id": "test_240517-2"}})
)
generate_confirm_message_chain_with_memory_lambda

RunnableLambda(lambda x: generate_confirm_message_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-2'}}))

일단 나중에 구현하기로 보류

In [ ]:
from langchain_core.runnables import RunnableLambda

def enrichMemoryContext(session_id, input, context, chain_with_memory):
    RunnableLambda(
        chain_with_memory.invoke(input,
                                 config={"configurable": {"session_id": session_id}})
                                 )
    

## 주문 조회 함수

In [ ]:
def fetch_recent_orders(dict):
    print(dict)
    return "produdct: 백설기 quantity: 2 price: 13,000"

## 승인 메시지 포함 여부에 따른 라우트

구 버전

In [ ]:
def execution_or_message_route(info):
    print("execution_or_message_route 함수로 전달된 데이터 -> ", info)
    if "yes" in info["execution_confirmation"].content.lower():
        return "주문 변경 또는 취소 진행"
    else:
        return RunnablePassthrough.assign(queried_result=fetch_recent_orders) | generate_confirm_message_chain

수정 버전
* generate_confirm_message_chain_with_memory_lambda 추가

In [291]:
def execution_or_message_route(info):
    print("execution_or_message_route 함수로 전달된 데이터 -> ", info)
    if "yes" in info["execution_confirmation"].content.lower():
        return "주문 변경 또는 취소 진행"
    else:
        return RunnablePassthrough.assign(queried_result=fetch_recent_orders) | generate_confirm_message_chain_with_memory_lambda

order_change_chain 동작 살펴보기 위해 임시 수정

In [304]:
def execution_or_message_route(info):
    print("execution_or_message_route 함수로 전달된 데이터 -> ", info)
    if "yes" in info["execution_confirmation"].content.lower():
        return order_change_chain_with_memory_lambda
    else:
        return RunnablePassthrough.assign(queried_result=fetch_recent_orders) | generate_confirm_message_chain_with_memory_lambda

In [305]:
execution_chain = confirmation_chain | execution_or_message_route
execution_chain

{
  inputs: RunnablePassthrough(),
  action_type: RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-2'}}))
}
| RunnableLambda(helper)
| RunnableAssign(mapper={
    execution_confirmation: RunnableLambda(lambda x: classify_confirmation_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-2'}}))
  })
| RunnableLambda(execution_or_message_route)

In [ ]:
response = execution_chain.invoke(
    {"input": None,
     "order_id": 7}
)
response

Parent run d66376b3-a137-4724-b6a2-81dcecf4f7d9 not found for run 9e7e916a-56c9-4feb-b7f8-f581f863ca62. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got None.')
Parent run de4989da-0619-480b-a754-a7f49249fd37 not found for run 805072c6-40c1-499c-8fd6-ba32501270b7. Treating as a root run.
Error in RootListenersTracer.on_chain_end callback: ValueError('Expected str, BaseMessage, List[BaseMessage], or Tuple[BaseMessage]. Got None.')


execution_or_message_route 함수로 전달된 데이터 ->  {'input': None, 'order_id': 7, 'action_type': AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 87, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5a4068af-71b3-4717-bc28-f36d058716fa-0'), 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 271, 'total_tokens': 272}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-85d2c44c-88d9-431a-8dce-06771bba1b64-0')}
{'input': None, 'order_id': 7, 'action_type': AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 87, 'total_tokens': 92}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5a4068af-71b3-4717-bc28-f36d058716fa-0'

AIMessage(content='주문 내역 확인 - 제품: 백설기, 수량: 2개, 가격: 13,000원\n주문을 취소하시겠습니까?', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 185, 'total_tokens': 236}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-11c03f97-76da-4f0e-88e9-4d32ee8e389f-0')

In [ ]:
store["test_240517-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-da099424-affc-4335-b10e-8c6bb4093e93-0')]

In [ ]:
response = execution_chain.invoke(
    {"input": "응",
     "order_id": 7}
)
response

Parent run 83b77107-a9ea-4bfd-8e6e-1254aad8a0ec not found for run f60e9d39-9b5a-4481-8102-84376e0e87ba. Treating as a root run.
Parent run 4e005ac1-ea5b-41b8-bb2f-a3b7eb458bb4 not found for run 0f75ec39-4ff4-41a3-9efa-00baa3357196. Treating as a root run.


execution_or_message_route 함수로 전달된 데이터 ->  {'input': '응', 'order_id': 7, 'action_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 86, 'total_tokens': 89}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dfc8a994-0e1a-48d7-aa26-71dfa5c23de1-0'), 'execution_confirmation': AIMessage(content='yes', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 284, 'total_tokens': 285}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b1569b47-166c-4969-a53e-ead7c9f5f1af-0')}


'주문 변경 또는 취소 진행'

In [320]:
response = execution_chain.invoke(
    {"input": '없음',
     "order_id": 3}
)
response

Parent run 4c281470-0b18-4e1b-b13b-20e5c14f29a1 not found for run 6694aff6-23af-4320-ac57-562af4c265e3. Treating as a root run.
Parent run 9db37b8e-6eaa-4739-8c81-3a0121ea67c2 not found for run 16ef045e-078e-4339-93d9-25c5febe9048. Treating as a root run.
Parent run 188072dd-027a-4275-81e2-94af2c670740 not found for run d6e7fd5b-3d68-4dc0-aceb-23f5acea619a. Treating as a root run.


execution_or_message_route 함수로 전달된 데이터 ->  {'input': '없음', 'order_id': 3, 'action_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 195, 'total_tokens': 198}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-703fcf12-aaef-4ac7-9f9c-e64fe176b0a8-0'), 'execution_confirmation': AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 286, 'total_tokens': 287}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-808420a1-40f8-44d6-8e9b-0f0c576466c1-0')}
{'input': '없음', 'order_id': 3, 'action_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 195, 'total_tokens': 198}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-703fcf12-aaef-4ac7-9f9c-e64fe176b0a

AIMessage(content='주문 내용을 확인해 주세요.\n\n상품: 백설기\n수량: 2\n가격: 13,000\n\n주문을 변경하시겠습니까? 최종 확인 부탁드립니다.', response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 186, 'total_tokens': 249}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b18fa125-014b-4304-a4e7-e005697dfff4-0')

이전 대화의 주문 변공하고 싶어 내용 보고도 멍청하게 '주문 취소'로 분류했던 적 있음

In [321]:
store["test_240517-2"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5af9ca04-d147-47a9-9c23-5f3f258128cb-0'),
 HumanMessage(content='없음'),
 AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 195, 'total_tokens': 198}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-703fcf12-aaef-4ac7-9f9c-e64fe176b0a8-0'),
 HumanMessage(content='없음'),
 AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 286, 'total_tokens': 287}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-808420a1-40f8-44d6-8e9b-0f0c576466c1-0'),
 HumanMessage(content='없음'),
 AIMessage(content='주문 내용을 확인해

In [322]:
response = execution_chain.invoke(
    {"input": '응',
     "order_id": 3}
)
response

Parent run c4506944-84df-467d-8f27-f28087251bc7 not found for run c5c82310-3888-4dae-9156-5932c3fba8c5. Treating as a root run.
Parent run dadcc70c-f762-4981-aa8d-94443d8ce5dc not found for run 181ca3b5-4dc7-41df-831a-26303ea8fe07. Treating as a root run.


execution_or_message_route 함수로 전달된 데이터 ->  {'input': '응', 'order_id': 3, 'action_type': AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 294, 'total_tokens': 297}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ad5ec986-90ab-43e0-b34b-ed2d453d4dd8-0'), 'execution_confirmation': AIMessage(content='yes', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 381, 'total_tokens': 382}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c16f070f-78fa-4100-8be2-9d2d742942dd-0')}


NameError: name 'order_change_chain_with_memory_lambda' is not defined

In [ ]:
store["test_240517-2"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8f04f2de-8dfe-4b6e-80d1-f30c081d2645-0'),
 HumanMessage(content='없음'),
 AIMessage(content='주문 변경', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 88, 'total_tokens': 91}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-52a346bd-e424-43f7-a671-cd6aa381f629-0'),
 HumanMessage(content='없음'),
 AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 281, 'total_tokens': 282}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8bf240bb-75c0-46a7-b9b9-d915c59359ce-0'),
 HumanMessage(content='없음'),
 AIMessage(content='주문 내용을 변경하려고

## 주문 취소 처리 체인

In [ ]:
def cancel_order(dict):
    order_id = dict["order_id"]
    return f"주문{order_id} 취소"

## 주문 변경 처리 체인

In [298]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a customer service assistant responsible for identifying and processing order changes based on customer chat history.
            First, find an AIMessage in the chat history that presents an 'order_id' and the corresponding order details, asking if the customer wishes to change the order.
            Then, look for the customer's response to the AIMessage regarding their desired changes. 
            Combine the AIMessage provided order details and the customer’s input to finalize the order changes.
            
            If the customer's response indicates that the presented order details are not the ones they want to change, do not proceed with the order change as it indicates a mismatch.
            """
        ),
        # MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

In [ ]:
order_change_chain = order_change_prompt | model
order_change_chain

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

order_change_chain_with_memory = RunnableWithMessageHistory(
    order_change_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
order_change_chain_with_memory_lambda = RunnableLambda(
    lambda x: order_change_chain_with_memory.invoke(x,
                                                    config={"configurable": {"session_id": "test_240517-2"}})
                                                    )

In [ ]:
store["test_240517-1"].messages

[HumanMessage(content='주문 취소하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 150, 'total_tokens': 153}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-da099424-affc-4335-b10e-8c6bb4093e93-0'),
 HumanMessage(content='없음'),
 AIMessage(content='주문 취소', response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 89, 'total_tokens': 94}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d3e2950d-3111-40bc-8ff3-7a1ac5996cf3-0'),
 HumanMessage(content='없음'),
 AIMessage(content='no', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 292, 'total_tokens': 293}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-0aeac858-9413-4f2a-821e-f779e312e018-0')]

In [ ]:
store["test_240517-2"].messages

[HumanMessage(content='주문 변경하고 싶어'),
 AIMessage(content='요청', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 148, 'total_tokens': 151}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9a2c12fc-1e47-42e4-8d65-b0513859cc59-0')]

## 주문 변경/취소 분류 체인

이젠 필요 없음

In [ ]:
chat_history = store["test_240516-1"].messages
chat_history 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            you need to review the chat_history from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
order_change_cancel_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            you need to review the chat_history from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])

프롬프트 수정
* You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt= ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
order_change_cancel_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n            ")), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input', 'order_id'], template='input:{input}\norder_id:{order_id}'))])

프롬프트 수정2

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

order_change_cancel_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            You are a robot that classifies customer input messages into specific types. 
            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.

            Classify as '주문 변경' if there are any indications or keywords related to modifying an order, such as:
            - 변경
            - 수정
            - 바꾸다
            - 변경하고 싶어

            Classify as '주문 취소' if there are any indications or keywords related to canceling an order, such as:
            - 취소
            - 취소하고 싶어
            - 취소 요청
            """
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "input:{input}\norder_id:{order_id}"),
    ]
)
order_change_cancel_prompt


ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n\n            Classify as '주문 변경' if there are any indications or keywords related to modifying an order, such as:\n            - 변경\n            - 수정\n            - 바꾸다\n            - 변경하고 싶어\n\n            Classify as '주문 취소' if there are any indications or keywords related to can

In [ ]:
classify_change_or_cancel_chain = order_change_cancel_prompt | model
classify_change_or_cancel_chain

ChatPromptTemplate(input_variables=['chat_history', 'input', 'order_id'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a robot that classifies customer input messages into specific types. \n            You need to review the messages in the Messages Placeholder from the latest to the oldest and output either '주문 변경' or '주문 취소'.\n\n            Classify as '주문 변경' if there are any indications or keywords related to modifying an order, such as:\n            - 변경\n            - 수정\n            - 바꾸다\n            - 변경하고 싶어\n\n            Classify as '주문 취소' if there are any indications or keywords related to can

오늘 추가 부분

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

classify_change_or_cancel_chain_with_memory = RunnableWithMessageHistory(
    classify_change_or_cancel_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

최초 지정 session_id

In [ ]:
from langchain_core.runnables import RunnableLambda
classify_change_or_cancel_chain_with_memory_lambda = RunnableLambda(
    lambda x: classify_change_or_cancel_chain_with_memory.invoke(x,
                                                                 config={"configurable": {"session_id": "test_240517-1"}})
)
classify_change_or_cancel_chain_with_memory_lambda

RunnableLambda(lambda x: classify_change_or_cancel_chain_with_memory.invoke(x, config={'configurable': {'session_id': 'test_240517-1'}}))

In [ ]:
order_change_chain = order_change_prompt | model
order_change_chain

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="\n            You are a customer service assistant responsible for identifying and processing order changes based on customer chat history.\n            First, find an AIMessage in the chat history that presents an 'order_id' and the corresponding order details, asking if the customer wishes to change the order.\n            Then, look for the customer's response to the AIMessage regarding their desired changes. \n            Combine the AIMessage provided order details and the customer’s input to finalize the order changes

## 주문 변경, 주문 취소 라우트

In [ ]:
def execution_route(info):
    print("execution_route 함수로 전달된 데이터 -> ", info)
    if "주문 변경" in info["action_type"].content:
        return 
    else:
        return cancel_order

In [ ]:
from langchain_core.messages import 